In [7]:
# Add MOSQITO to the Python path
import os
import sys
sys.path.append('..')

# Import numpy
import numpy as np
# Import plot function
import matplotlib.pyplot as plt
# Import spectrum computation tool
from scipy.fft import fft, fftfreq
from scipy.signal import stft
# Import mosqito functions
from mosqito.utils import load

# LOUDNESS_ZWST functions
from mosqito.sq_metrics import loudness_zwst
from mosqito.sq_metrics import loudness_zwst_perseg
from mosqito.sq_metrics import loudness_zwst_freq


# ROUGHNESS_DW functions
from mosqito.sq_metrics import roughness_dw, roughness_dw_freq


# SHARPNESS_DIN functions
from mosqito.sq_metrics import sharpness_din_st
from mosqito.sq_metrics import sharpness_din_perseg
from mosqito.sq_metrics import sharpness_din_from_loudness
from mosqito.sq_metrics import sharpness_din_freq


# Import MOSQITO color sheme [Optional]
from mosqito import COLORS

# To get inline plots (specific to Jupyter notebook)
# %matplotlib notebook

In [ ]:
LOUDNESS_ZWST_STRING = "LOUDNESS_ZWST"
ROUGHNESS_DW_STRING = "ROUGHNESS_DW"
SHARPNESS_DIN_STRING = "SHARPNESS_DIN"

GENERAL_FOLDERS = [LOUDNESS_ZWST_STRING, ROUGHNESS_DW_STRING, SHARPNESS_DIN_STRING]

In [9]:
def mkdir_folder_wav_name(output_path, wav_name, png_string, folder_string):
    output_path_folder = os.path.join(output_path, f'{folder_string}')
    print(f"The output path for the signal plot is: {output_path_folder}")
    if not os.path.exists(output_path_folder):
        os.makedirs(output_path_folder)
        print(f"The output signal folder was created: {output_path_folder}")
    
    
    #adding the output_path_folder}") + wav_name + '_signal.png' to save the signal plot in the correct folder
    wav_name_path = os.path.join(output_path_folder, wav_name + f'{png_string}')
    print(f"The output path for the signal plot is: {wav_name_path}")

    
    return output_path_folder, wav_name_path

In [ ]:
# Define path to the .wav file
folder_path = r"\\192.168.205.115\aac_server\NoiseTech\3-Medidas\FASE_2"
mision_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]


for folder in mision_folders:
    mission_path = os.path.join(folder_path, folder)

    #checking if they exit
    if os.path.exists(mission_path):
        # print(f"The folder exists: {mission_path}")


        for general_folder in GENERAL_FOLDERS:

            #5-RESULT FOLDER
            output_path = mission_path.replace("3-Medidas", "5-Resultados")
            if not os.path.exists(output_path):
                os.makedirs(output_path)
                print(f"The output folder was created: {output_path}")
            else:
                print(f"The output folder already exists: {output_path}")
            print()


            # GENERAL FOLDER (LOUDNESS_ZWST, ROUGHNESS_DW, SHARPNESS_DIN)
            general_analysis_folder = os.path.join(output_path, f'{general_folder}')
            print(f"The output path is: {general_analysis_folder}")

            if not os.path.exists(general_analysis_folder):
                os.makedirs(general_analysis_folder)
                print(f"The output folder was created: {general_analysis_folder}")
            else:
                print(f"The output folder already exists: {general_analysis_folder}")
            print()


            wav_files = [f for f in os.listdir(mission_path) if f.endswith('.wav')]
            for wav_file in wav_files:
                file_path = os.path.join(mission_path, wav_file)
                wav_name = os.path.splitext(wav_file)[0]

                # load signal
                sig, fs = load(file_path, wav_calib=2 * 2 **0.5)
                # plot signal
                t = np.linspace(0, (len(sig) - 1) / fs, len(sig))



                if general_folder == LOUDNESS_ZWST_STRING:
                    print()
                    print("ENTERING THE LOUDNESS_ZWST BLOCK")
                    _, loudness_zwst_signal_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_signal.png', 'Signal')
                    _, loudness_zwst_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_loudness_zwst.png', 'Loudness_Zwicker')
                    _, loudness_zwst_band_rate_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_loudness_zwst_band_rate.png', 'Loudness_Zwicker_Band_Rate')
                    _, loudness_zwst_segment_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_loudness_zwst_segment.png', 'Loudness_Zwicker_Segment')            
                    _, loudness_zwst_spectrum_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_loudness_zwst_spectrum.png', 'Loudness_Zwicker_Spectrum')



                    ###################################
                    ###################################
                    ###################################
                    ## SIGNAL ##
                    ###################################
                    ###################################
                    ###################################
                    plt.figure(1)
                    plt.plot(t, sig, color=COLORS[0])
                    plt.xlabel('Time [s]')
                    plt.ylabel('Acoustic pressure [Pa]')
                    plt.title(f'Signal - {wav_name}')
                    plt.grid()

                    #SAVE
                    plt.savefig(loudness_zwst_signal_wav_name, dpi=300)
                    print(f"Signal plot saved: {loudness_zwst_signal_wav_name}")
                    plt.close()
                    # break




                    # # ###################################
                    # # ###################################
                    # # ###################################
                    # # # COMPUTTING LOUDNESS ZWST
                    # # ###################################
                    # # ###################################
                    # # ###################################
                    N, N_specific, bark_axis = loudness_zwst(sig, fs, field_type="free")
                    print("N_zwst = {:.1f} sone".format(N))
                    print(f"N_zwst = {N:.1f} sone")

                    plt.figure(2)
                    plt.plot(bark_axis, N_specific, color=COLORS[0])
                    plt.xlabel("Critical band rate [Bark]")
                    plt.ylabel("N'_zwst [sone/Bark]")
                    plt.title(f"Loudness Zwicker - N_zwst = {N:.1f} sone - {wav_name}".format(N))
                    plt.grid()

                    # #SAVE
                    plt.savefig(loudness_zwst_wav_name, dpi=300)
                    print(f"Loudness Zwicker plot saved: {loudness_zwst_wav_name}")
                    print()
                    plt.close()
                    



                    # ###################################
                    # ###################################
                    # ###################################
                    # # # SPECIFIC LOUDNESS OVER CRITICAL BAND RATE IS ALSO COMPUTED
                    # ###################################
                    # ###################################
                    # ###################################
                    plt.figure(2)
                    plt.plot(bark_axis, N_specific, color=COLORS[0])
                    plt.xlabel("Critical band rate [Bark]")
                    plt.ylabel("N'_zwst [sone/Bark]")
                    plt.title(f"Loudness Zwicker - N_zwst = {N:.1f} sone - {wav_name}".format(N))
                    plt.grid()

                    # #SAVE
                    plt.savefig(loudness_zwst_band_rate_wav_name, dpi=300)
                    print(f"Loudness Zwicker Band Rate plot saved: {loudness_zwst_band_rate_wav_name}")
                    print()
                    plt.close()



                    # ###################################
                    # ###################################
                    # ###################################
                    # # # SPECIFIC LOUDNESS PER SEGMENT
                    # ###################################
                    # ###################################
                    # ###################################
                    N, N_specific, bark_axis, time_axis = loudness_zwst_perseg(
                        sig, fs, nperseg=8192 * 2, noverlap=4096
                    )
                    plt.figure(3)
                    plt.plot(time_axis, N, color=COLORS[0])
                    plt.xlabel("Time [s]")
                    plt.ylabel("N_zwst [sone]")
                    # plt.ylim((0, 15))
                    plt.title(f"Loudness Zwicker - {wav_name}")
                    plt.grid()

                    # # save
                    plt.savefig(loudness_zwst_segment_wav_name, dpi=300)
                    print(f"Loudness Zwicker Segment plot saved: {loudness_zwst_segment_wav_name}")
                    print()
                    plt.close()


                    # ###################################
                    # ###################################
                    # ###################################
                    # # # FROM SPECTRUM
                    # ###################################
                    # ###################################
                    # ######################################################################
                    # ###################################
                    # ###################################
                    # # # Compute spectrum
                    n = len(sig)
                    spec = np.abs(2 / np.sqrt(2) / n * fft(sig)[0:n//2])
                    freqs = fftfreq(n, 1/fs)[0:n//2]
                    # Compute Loudness
                    N, N_specific, bark_axis = loudness_zwst_freq(spec, freqs)

                    print("N_zwst = {:.1f} sone".format(N) )

                    # Plot the results
                    plt.figure(6)
                    plt.plot(bark_axis, N_specific, color=COLORS[0])
                    plt.xlabel("Critical band rate [Bark]")
                    plt.ylabel("N'_zwst [sone/Bark]")
                    plt.title(f"Loudness Zwicker from spectrum - N_zwst = {N:.1f} sone - {wav_name}".format(N))
                    plt.grid()
                    #save
                    plt.savefig(loudness_zwst_spectrum_wav_name, dpi=300)
                    print(f"Loudness Zwicker Spectrum plot saved: {loudness_zwst_spectrum_wav_name}")
                    print()
                    plt.close()



                if general_folder == ROUGHNESS_DW_STRING:
                    print("ENTERING THE LOUDNESS_ZWST BLOCK")
                    print()
                    _, roughness_dw_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_roughness_dw.png', 'Roughness')
                    _, roughness_dw_spectrum_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_roughness_dw_spectrum.png', 'Roughness_Spectrum')



                    # ###################################
                    # ###################################
                    # ###################################
                    # # COMPUTTING ROUGHNESS DW
                    # ###################################
                    # ###################################
                    # ###################################
                    r, r_spec, bark, time = roughness_dw(sig, fs, overlap=0)

                    plt.figure(2)
                    plt.plot(time, r, color=COLORS[0])
                    # plt.ylim(0,1.5) # modifying this to plot the whole signal
                    plt.xlabel("Time [s]")
                    plt.ylabel("Roughness [Asper]")
                    plt.title(f'Roughness - {wav_name}')
                    plt.grid()

                    #save
                    plt.savefig(roughness_dw_wav_name, dpi=300)
                    print(f"Roughness plot saved: {roughness_dw_wav_name}")
                    plt.close()



                    ###################################
                    ###################################
                    ###################################
                    # # FROM SPECTRUM
                    ###################################
                    ###################################
                    ######################################################################
                    ###################################
                    ###################################
                    # Compute multiple spectra along time
                    freqs, time, spectrum = stft(sig, fs=fs)

                    # Compute roughness
                    R, R_spec, bark = roughness_dw_freq(spectrum,freqs)

                    # Plot the results
                    plt.figure(6)
                    plt.plot(time, R, color=COLORS[0])
                    # plt.ylim(0,1)
                    plt.xlabel("Time [s]")
                    plt.ylabel("Roughness [Asper]")
                    plt.title(f'Roughness from spectrum - {wav_name}')
                    plt.grid()

                    #save
                    plt.savefig(roughness_dw_spectrum_wav_name, dpi=300)
                    print(f"Roughness Spectrum plot saved: {roughness_dw_spectrum_wav_name}")
                    print()
                    plt.close()




                if general_folder == SHARPNESS_DIN_STRING:
                    _, sharpness_din_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_sharpness_din.png', 'Sharpness_Din')
                    _, sharpness_din_spectrum_wav_name = mkdir_folder_wav_name(general_analysis_folder, wav_name, '_sharpness_din_spectrum.png', 'Sharpness_Din_Spectrum')



                    # ###################################
                    # ###################################
                    # ###################################
                    # # COMPUTTING ROUGHNESS DW
                    # ###################################
                    # ###################################
                    # ###################################
                    sharpness = sharpness_din_st(sig, fs, weighting="din")
                    # sharpness = sharpness_din_st(sig, fs, weighting="aures")
                    # sharpness = sharpness_din_st(sig, fs, weighting="fastl")

                    print("Sharpness = {:.1f} acum".format(sharpness) )
                    sharpness_n = "Sharpness = {:.1f} acum".format(sharpness)
                    print(sharpness_n)

                    sharpness, time_axis = sharpness_din_perseg(sig, fs, nperseg=8192 * 2, noverlap=4096, weighting="din")
                    # sharpness, time_axis = sharpness_din_perseg(sig, fs, nperseg=8192 * 2, noverlap=4096, weighting="aures")
                    # sharpness, time_axis = sharpness_din_perseg(sig, fs, nperseg=8192 * 2, noverlap=4096, weighting="fastl")
                    plt.figure(2)
                    plt.plot(time_axis, sharpness, color=COLORS[0])
                    plt.xlabel("Time [s]")
                    plt.ylabel("S_din [acum]")
                    # plt.ylim((0, 3))
                    plt.title(f'Sharpness - {sharpness_n} - {wav_name}')
                    plt.grid()

                    # # save
                    plt.savefig(sharpness_din_wav_name, dpi=300)
                    print(f"Sharpness plot saved: {sharpness_din_wav_name}")
                    print()
                    plt.close()



                    ###################################
                    ###################################
                    ###################################
                    # # FROM SPECTRUM
                    ###################################
                    ###################################
                    ######################################################################
                    ###################################
                    ###################################
                    # Compute spectrum
                    n = len(sig)
                    spec = np.abs(2 / np.sqrt(2) / n * fft(sig)[0:n//2])
                    freqs = fftfreq(n, 1/fs)[0:n//2]
                    # Compute sharpness
                    S = sharpness_din_freq(spec, freqs)
                    print("Sharpness_din = {:.1f} sone".format(S) )

                    # Plot the results
                    plt.figure(6)
                    plt.bar(1, S, color=COLORS[0])
                    # plt.xlim(0,2)
                    # plt.ylim(0,3)
                    plt.xlabel("Sharpness [acum]")
                    plt.title(f'Sharpness from spectrum - {wav_name}')
                    plt.grid()

                    #save
                    plt.savefig(sharpness_din_spectrum_wav_name, dpi=300)
                    print(f"Sharpness Spectrum plot saved: {sharpness_din_spectrum_wav_name}")
                    plt.close()


The output folder already exists: \\192.168.205.115\aac_server\NoiseTech\5-Resultados\FASE_2\Mision_1

The output path is: \\192.168.205.115\aac_server\NoiseTech\5-Resultados\FASE_2\Mision_1\LOUDNESS_ZWST
The output folder already exists: \\192.168.205.115\aac_server\NoiseTech\5-Resultados\FASE_2\Mision_1\LOUDNESS_ZWST

[Info] Multichannel signal loaded. Keeping only first channel

ENTERING THE LOUDNESS_ZWST BLOCK
The output path for the signal plot is: \\192.168.205.115\aac_server\NoiseTech\5-Resultados\FASE_2\Mision_1\LOUDNESS_ZWST\Signal
The output path for the signal plot is: \\192.168.205.115\aac_server\NoiseTech\5-Resultados\FASE_2\Mision_1\LOUDNESS_ZWST\Signal\fronton_SanNicolas_signal.png
The output path for the signal plot is: \\192.168.205.115\aac_server\NoiseTech\5-Resultados\FASE_2\Mision_1\LOUDNESS_ZWST\Loudness_Zwicker
The output path for the signal plot is: \\192.168.205.115\aac_server\NoiseTech\5-Resultados\FASE_2\Mision_1\LOUDNESS_ZWST\Loudness_Zwicker\fronton_SanNicol